In [1]:
%load_ext autoreload

In [26]:
from tesspoint import TESSPoint, footprint
import numpy as np

In [29]:
from tesspoint.tesspoint import sphereToCart, cartToSphere

In [8]:
self = TESSPoint(1, 1, 1)

In [12]:
coords = self.pix2radec(footprint())

In [18]:
vecs = sphereToCart(*coords)

In [108]:
self.opt_coeffs

array([145.948116  ,   1.0000014 ,   0.24779006,  -0.22681254,
        10.78243356, -34.97817276])

In [ ]:
def optics_fp(self, lng, lat):
    thetar = np.pi / 2.0 - np.deg2rad(lat_deg)
    tanth = np.tan(thetar)
    cphi = np.cos(np.deg2rad(lng)
    sphi = np.sin(np.deg2rad(lng)
    rfp0 = self.optcon[icam][0]*tanth
    noptcon = len(self.optcon[icam])
    ii = np.arange(1, noptcon)
    rfp = np.sum(self.optcon[icam][1:] * np.power(tanth, 2.0*(ii-1)))
    xytmp = np.zeros((2,), dtype=np.double)
    xytmp[0] = -cphi*rfp0*rfp
    xytmp[1] = -sphi*rfp0*rfp
    return self.make_az_asym(icam, xytmp)

In [102]:
def star_in_fov(lng, lat):
    vec = np.asarray(sphereToCart(lng, lat))
    norm = np.sqrt(np.sum(vec**2, axis=0))
    vec = vec / norm
    xlen = np.abs(np.arctan(vec[0]/vec[2]))
    ylen = np.abs(np.arctan(vec[1]/vec[2]))
    inview = (xlen <= np.deg2rad(12.5)) & (ylen <= np.deg2rad(12.5))
    return inview

In [61]:
j = 0
# Do the rotation from ra dec coords to camera coords
camVec = np.matmul(self.rmat4, vecs)
lng, lat = np.deg2rad(cartToSphere(camVec.T))

(200,)

In [ ]:
def radec2pix(self, coords):
    """ After the rotation matrices are defined to the actual
        ra and dec to pixel coords mapping
    """
    if self.havePointing == True:
        # Convert ra and dec spherical coords to cartesian
        vecs = sphereToCart(*coords)
        for i in range(nStar):
            curVec = np.array([vec0s[i], vec1s[i], vec2s[i]], dtype=np.double)
            # Find the new vector in all cameras
            for j in range(self.NCAM):
                # Do the rotation from ra dec coords to camera coords
                camVec = np.matmul(self.rmat4[j], curVec)
                # Get the longitude and latitude of camera coords position
                lng, lat = self.cartToSphere(camVec)
                lng = lng / deg2rad
                lat = lat / deg2rad
                if self.star_in_fov(lng, lat):
                    # Get the xy focal plane position in mm
                    xyfp = self.optics_fp(j, lng, lat)
                    # Convert mm to pixels
                    iccd, ccdpx, fitpx = self.mm_to_pix(j, xyfp)
                    inCamera = np.append(inCamera, j+1) # Als code is base 0 convert to base 1
                    ccdNum = np.append(ccdNum, iccd+1) # ""
                    fitsxpos = np.append(fitsxpos, fitpx[0])
                    fitsypos = np.append(fitsypos, fitpx[1])
                    ccdxpos = np.append(ccdxpos, ccdpx[0])
                    ccdypos = np.append(ccdypos, ccdpx[1])

    else:
        print('Spacecraft Pointing Not specified!')
